# Scalable Quantum Computing Experiment
This notebook contains code for conducting a scalable quantum computing experiment using Qiskit and other scientific libraries.

In [ ]:

# Import necessary libraries
import itertools as it
import numpy as np
import functools as ft
import operator
from numpy.linalg import norm
import sympy as sp
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, transpile
from qiskit.quantum_info import Statevector, random_statevector
from qiskit.quantum_info import state_fidelity as fid
from tqdm import tqdm
import qutip as qt
import pandas as pd
import time


In [ ]:

# Function to convert decimal to binary as an array
def Dec2Bin(x, n):
    # Convert the decimal number 'x' to a binary representation with 'n' bits
    return np.array([int(i) for i in np.binary_repr(x, width=n)])


In [ ]:

# Function to get projectors for a given number of qubits 'n'
def get_proj(n):
    circ_idx = {}  # Dictionary to store projectors for each qubit
    T_Proj = []  # Temporary list to store projectors
    states = ['0', '1']  # Basis states
    idx_r = [0, 1]  # Indices for the basis states

    # Iterate over each qubit
    for j in range(n):
        z_len = n - (j + 1)  # Length of the remaining qubits
        b_len = 2**z_len - 1  # Number of combinations for remaining qubits
        m_len = (j + 1) - 1  # Length of measured qubits

        # Generate projectors for each combination
        if b_len != 0:
            for b in range(b_len + 1):
                proj = []
                Bin = Dec2Bin(b, z_len)  # Convert to binary representation
                for i in Bin:
                    proj.append(states[i])  # Append basis state
                proj.append(states[idx_r[0]])  # Append measurement state
                for i in range(m_len):
                    proj.append(states[idx_r[1]])  # Append remaining states
                T_Proj.append(''.join(proj))
        else:
            proj = [states[idx_r[0]]]  # Initialize projector with measurement state
            for i in range(m_len):
                proj.append(states[idx_r[1]])  # Append remaining states
            T_Proj.append(''.join(proj))

        circ_idx[j] = T_Proj  # Store projectors for the current qubit
        T_Proj = []  # Reset temporary list for next qubit

    return circ_idx  # Return dictionary of projectors


In [ ]:

# Main experimental setup
def main_experiment():
    n = 2  # Number of qubits
    d = 2**n  # Dimension of the state space
    avg = 100  # Number of averages for the experiment
    Fid, Time, avg_fid, avg_time, cop = [], [], [], [], []  # Lists to store results
    po = 5  # Number of different shot settings

    # Loop over different shot settings
    for c in range(po):
        shots = 100000 * (10**c)  # Number of copies (shots) for measurement
        for k in tqdm(range(avg)):
            # Generate a random state using Haar measure
            psi = np.array(qt.rand_ket_haar(d).full()).flatten()
            rho = np.outer(psi, psi.conj())  # Density matrix of the state
            PSI = Statevector(psi)  # Create a Qiskit Statevector object

            # Estimation process
            st_sca = time.time()  # Start time for the scalable estimation
            prob_dict_1 = {}  # Placeholder for measurement probabilities for setting 1
            prob_dict_2 = {}  # Placeholder for measurement probabilities for setting 2
            prob_comp = {}  # Placeholder for complete measurement probabilities
            rho_est = rho  # Placeholder for estimated density matrix
            end_sca = time.time()  # End time for the scalable estimation

            # Calculate infidelity and timing
            Fid.append(1 - fid(psi, rho_est))  # Calculate infidelity between the original and estimated state
            Time.append(end_sca - st_sca)  # Calculate time taken for estimation

        # Store median infidelity and time for the current shot setting
        avg_fid.append(np.median(Fid))
        avg_time.append(np.median(Time))
        print(f'Average Infidelity: {np.median(Fid)}')  # Print average infidelity
        print(f'Average Time: {np.median(Time)}')  # Print average time
        cop.append(shots)  # Store the number of copies (shots)

    # Save results to CSV files
    fid_dict = {'Copies': cop, 'SCA': np.array(avg_fid)}  # Dictionary for infidelity results
    time_dict = {'Copies': cop, 'SCA': np.array(avg_time)}  # Dictionary for timing results
    df_fid = pd.DataFrame(fid_dict)  # Create DataFrame for infidelity results
    df_time = pd.DataFrame(time_dict)  # Create DataFrame for timing results
    df_fid.to_csv(f'{n}_infid_SCA.csv', index=False)  # Save infidelity results to CSV
    df_time.to_csv(f'{n}_time_SCA.csv', index=False)  # Save timing results to CSV

# Run the main experiment if this script is executed
if __name__ == "__main__":
    main_experiment()
